In [ ]:
import glob

import pandas as pd
from llavaguard.taxonomy.PEGI.PEGI_Graph import get_content_categories, get_content_categories_with_numbers, policy_graph
from PIL import Image
from llavaguard.taxonomy.policy_config import local_image_dirs, local_data_dir
from llavaguard.taxonomy.PEGI.PEGI_Graph import get_policy_intro, policy_graph_to_text
import random
from itertools import product
import os
import pandas as pd
from llavaguard.taxonomy.PEGI.PEGI_Graph import remove_mutually_exclusive_categories



# User study - find percentage of correct labels for a small subset

In [ ]:
pd.read_csv(f"{local_data_dir}/data/annotations/auto_generated_annotations/combined_annotations_extracted_cleaned.csv")

In [ ]:
# First cell - imports
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from PIL import Image

# Second cell - load data
data = pd.read_csv(f"{local_data_dir}/data/annotations/auto_generated_annotations/combined_annotations_extracted_cleaned.csv")
data = data[data['voting_mechanism'] == 'majority_vote']
data = data.iloc[:200]
categories = [col for col in data.columns if col not in ['voting_mechanism', 'sample_id', 'im_path']]

# Third cell - UI class
class NotebookAnnotationUI:
    def __init__(self, data, categories):
        self.data = data
        self.categories = categories
        self.current_idx = 0
        self.human_feedback = pd.DataFrame(columns=['sample_id'] + categories)
        
        # Create widgets
        self.image_widget = widgets.Image(format='png', width=400, height=400)
        self.output = widgets.Output()
        self.checkbox_widgets = {
            category: widgets.Checkbox(description=category, value=False, indent=False, layout=widgets.Layout(margin='0px 10px 0px 0px'))
            for category in categories
        }
        self.next_button = widgets.Button(description='Next Image')
        self.next_button.on_click(self.next_image)
        # Create rows of 4 checkboxes
        checkbox_rows = []
        for i in range(0, len(categories), 4):
            row_widgets = list(self.checkbox_widgets.values())[i:i+4]
            row = widgets.HBox(row_widgets)
            checkbox_rows.append(row)
        # Display widgets
        self.widget_box = widgets.VBox([
            self.output,
            widgets.VBox(checkbox_rows, layout=widgets.Layout(margin='0px 0px')),  # Stack rows vertically
            self.next_button
        ])
        display(self.image_widget)
        display(self.widget_box)
        self.load_current_image()


    def load_current_image(self):
        if self.current_idx < len(self.data):
            row = self.data.iloc[self.current_idx]
            image = Image.open(row['im_path']).resize((400, 400))
            # Convert PIL image to bytes for widget
            import io
            buf = io.BytesIO()
            image.save(buf, format='PNG')
            self.image_widget.value = buf.getvalue()            
            # Set checkbox values
            for category in self.categories:
                self.checkbox_widgets[category].value = bool(row[category])
    
    def next_image(self, b):
        self.save_current_feedback()
        self.current_idx += 1
        if self.current_idx < len(self.data):
            self.load_current_image()
        else:
            self.human_feedback.to_csv(f'{local_data_dir}/data/annotations/auto_generated_annotations/human_feedback.csv', index=False)
            clear_output()
            print("Annotation complete! Results saved to human_feedback.csv")
    
    def save_current_feedback(self):
        if self.current_idx < len(self.data):
            feedback = {'sample_id': self.data.iloc[self.current_idx]['sample_id']}
            for category in self.categories:
                feedback[category] = self.checkbox_widgets[category].value
            self.human_feedback = pd.concat([
                self.human_feedback,
                pd.DataFrame([feedback])
            ], ignore_index=True)

# Fourth cell - create and run UI
ui = NotebookAnnotationUI(data, categories)

## impressions
####  impression -> highly disturbing content is overly presset maybe rephrase description to only include horriphic scenes
####  eductional content. Only when it is content clearly of educational nature such as general information, statistics, school books, etc.

In [ ]:
human_feedback = pd.read_csv('human_feedback.csv')
model_data = pd.read_csv(f"{local_data_dir}/data/annotations/auto_generated_annotations/combined_annotations_extracted_cleaned.csv")

